In [ ]:
!python -m pip install -r ../requirements.txt

In [ ]:
import pdf2image
from PIL import Image
import pytesseract

In [ ]:
import os

if "OS" in os.environ.keys() and "WINDOWS" in os.environ["OS"].upper():
    OS="WINDOWS"
    pytesseract.pytesseract.tesseract_cmd = (
        r"C:\Users\conrad.thiounn\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
    )
    POPPLER_PATH = r"C:\Users\conrad.thiounn\Desktop\poppler-23.01.0\Library\bin"
else:
    OS="UNIX"
    pytesseract.pytesseract.tesseract_cmd = (
        r"/usr/bin/tesseract"
    )
    POPPLER_PATH = r"/usr/share/poppler"

In [ ]:
# https://stackoverflow.com/questions/64048828/pytesseract-gives-an-error-permissionerror-winerror-5-access-is-denied
def pdf_to_img(pdf_file):
    return pdf2image.convert_from_path(pdf_file,
        grayscale=True,thread_count=os.cpu_count())

def ocr_core(file,lang,config):
    if lang:
        text = pytesseract.image_to_string(file, lang=lang,config=config)
    else:
        text = pytesseract.image_to_string(file,config=config)
    return text

def print_pages(pdf_file,lang, config):
    txt = ""
    images = pdf_to_img(pdf_file)
    for pg, img in enumerate(images):
        txt += ocr_core(img,lang, config)
    return txt

def read_and_print_pdf_with_ocr(pdf_file_path: str,lang:str=None) -> None:
    custom_oem_psm_config = r"-c tessedit_do_invert=0  --oem 2 --psm 4 tessedit_char_whitelist=abcdefghijklmnopqrstuvwxyzàâéêèùçîô0123456789"


    return print_pages(pdf_file_path,lang, custom_oem_psm_config)

In [ ]:
PATH_TO_ALL_PDF="C:\\Users\\conrad.thiounn\\Desktop\\daccord-scraper\\Accords\\"
OUTPUT_DIR="C:\\Users\\conrad.thiounn\\Desktop\\ocr2\\"

In [ ]:
list_txt_ocr=[file_name for file_name in os.listdir(OUTPUT_DIR)]
for pdf in [file_name for file_name in os.listdir(PATH_TO_ALL_PDF) if ".pdf" in file_name]:
    new_file=pdf.replace(".pdf",".txt")
    if new_file not in list_txt_ocr:
        try:
            with open(OUTPUT_DIR+new_file, "w") as f:
                f.write(read_and_print_pdf_with_ocr(PATH_TO_ALL_PDF+pdf,lang="fra"))
        except:
            print(f"bug for {new_file}")